In [1]:
import pandas as pd
import cryptpandas as crp

decrypted_df = crp.read_encrypted(path='encrypted_data/release_3547.crypt', password='oUFtGMsMEEyPCCP6')

decrypted_df.to_csv('decrypted_data/release_3547.csv')

ModuleNotFoundError: No module named 'cryptpandas'

In [2]:
pip install cryptpandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
df = pd.read_csv("decrypted_data/release_3547.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [11]:
df

,strat_0,strat_1,strat_2,strat_3,strat_4,strat_5,strat_6,strat_7,strat_8,strat_9,...,strat_12,strat_13,strat_14,strat_15,strat_16,strat_17,strat_18,strat_19,strat_20,strat_21
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.216100,NaN,NaN,NaN,NaN,NaN
1,-1.045104,NaN,NaN,NaN,NaN,0.139780,1.481773,NaN,1.298774,NaN,...,NaN,NaN,NaN,-3.467195,1.344721,-0.175514,NaN,NaN,NaN,-0.827335
2,-0.229267,NaN,NaN,NaN,NaN,0.109780,0.731916,NaN,1.748748,NaN,...,NaN,NaN,NaN,-1.709005,0.681587,-0.235082,-1.271263,NaN,NaN,-0.121570
3,-0.332475,NaN,-0.056052,NaN,NaN,-0.038044,-0.557418,-0.246940,0.043789,NaN,...,NaN,NaN,NaN,0.506470,-0.403906,0.249474,-0.151417,-0.253624,NaN,-1.002791
4,-0.827446,NaN,-0.349031,NaN,NaN,-0.035166,-1.177777,-0.795178,-1.105112,NaN,...,NaN,NaN,NaN,0.370537,-0.813284,0.143559,0.391880,-0.888127,NaN,0.233682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3543,-0.381309,-0.212961,0.501306,0.322370,-0.103106,-0.557941,-0.020194,-0.008013,-0.355781,-0.881419,...,0.096630,0.679358,0.193971,-0.011222,-0.224635,-0.192642,0.206517,0.198630,0.295600,1.135041
3544,0.620255,0.064530,0.391176,-0.679921,-1.709302,1.486581,0.246001,-0.131926,0.878312,0.535155,...,-0.538533,0.895913,0.201795,0.012643,0.309842,-0.007732,-0.657715,-0.205288,-0.786703,-0.074781
3545,0.224598,0.413439,0.376754,-1.812926,-1.050624,0.827850,0.575480,-1.391991,0.693199,0.626953,...,0.822201,1.718395,-0.151103,-0.000421,0.083085,-0.103815,-0.328812,-1.100259,-0.219447,-1.173045
3546,-0.519673,-0.923652,-0.969943,-1.200744,-1.621024,-0.776032,-1.095705,0.621262,-0.606028,-1.556457,...,0.139280,-0.344232,0.560597,-0.006781,-0.723691,0.092065,0.281789,0.697799,0.301284,0.997957


In [12]:
import pandas as pd
import numpy as np


sharpe_ratios = {col: df[col].mean() / df[col].std() for col in df.columns}

sharpe_df = pd.DataFrame.from_dict(sharpe_ratios, orient='index', columns=['Sharpe Ratio']).T


In [13]:
sharpe_df

,strat_0,strat_1,strat_2,strat_3,strat_4,strat_5,strat_6,strat_7,strat_8,strat_9,...,strat_12,strat_13,strat_14,strat_15,strat_16,strat_17,strat_18,strat_19,strat_20,strat_21
Sharpe Ratio,0.020423,-0.086943,0.007065,0.058292,0.075674,0.028916,0.02736,0.039377,0.030454,0.006179,...,0.020486,-0.055378,-0.014605,0.005407,0.005928,0.032696,-0.012789,0.042522,-0.017805,0.093801


In [22]:
from scipy.optimize import minimize

# Clean the data by filling NaN values (e.g., using mean or forward-fill)
df_cleaned = df.fillna(df.mean())

# Recalculate mean returns and covariance matrix
mean_returns = df_cleaned.mean()
cov_matrix = df_cleaned.cov()

# Define the number of strategies
num_strategies = len(mean_returns)
epsilon = 1e-6
# Objective function (negative Sharpe ratio with covariance matrix)
def sharpe_objective(weights):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -portfolio_return / (portfolio_std + epsilon)  # Add epsilon to prevent division by zero

# Custom constraint for at least 10 non-zero weights
def non_zero_weights_constraint(weights):
    return np.sum(weights != 0) - 10  # Ensure at least 10 weights are non-zero

# Define constraints
constraints = [
    {"type": "eq", "fun": lambda w: np.sum(np.abs(w)) - 1},  # Absolute sum of weights = 1
    {"type": "eq", "fun": lambda w: np.sum(w)},  # Sum of weights = 0
    {"type": "ineq", "fun": non_zero_weights_constraint},  # At least 10 non-zero weights
]

# Bounds: Each weight between -0.1 and 0.1
bounds = [(-0.1, 0.1) for _ in range(num_strategies)]

# Initial guess for weights
initial_weights = np.random.uniform(-0.1, 0.1, num_strategies)

# Perform optimization
result_with_correlation = minimize(
    sharpe_objective,
    initial_weights,
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
)

# Extract the optimized weights
optimized_weights_with_correlation = result_with_correlation.x

# Convert to a DataFrame for better presentation
optimized_weights_with_correlation_df = pd.DataFrame(
    optimized_weights_with_correlation, index=df.columns, columns=["Optimized Weights"]
)
optimized_weights_with_correlation_df

,Optimized Weights
strat_0,-0.007922
strat_1,-0.100000
strat_2,-0.015617
strat_3,0.011628
strat_4,0.053937
strat_5,-0.005840
strat_6,0.026254
strat_7,0.074277
strat_8,-0.033935
strat_9,0.026118
